In [1]:
#Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
import time
import datetime
import re

#from flask import Flask, render_template, redirect
#from flask_pymongo import PyMongo
# import mission_to_mars


In [2]:
#URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url='https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
html=browser.html
#print(html)

In [5]:
#Create BeautifulSoup object; parse with 'html.parser'
soup=BeautifulSoup(html, 'html.parser')
#print(soup)

## NASA Mars News

In [6]:
#Pull the latest news article
news_slide = soup.select_one('ul.item_list li.slide')
#print(news_slide)

In [7]:
#Get latest news article title
news_title = news_slide.find("div", class_='content_title').get_text()
print(news_title)

After a Reset, Curiosity Is Operating Normally


In [8]:
#Get latest news article paragraph
news_p= news_slide.find('div', class_="article_teaser_body").get_text()
print(news_p)

NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. 


## JPL Mars Space Images

In [9]:
#Launch image url
img_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(img_url)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)
browser.click_link_by_partial_text('more info')

In [10]:
#Find the image URL for the current Featured Mars Image
html=browser.html
soup_img=BeautifulSoup(html,'html.parser')
fig=soup_img.find('figure', class_='lede')
partial_img=fig.find('a')['href']
featured_image_url = 'https://www.jpl.nasa.gov' + partial_img
print(featured_image_url)
browser.quit()


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17661_hires.jpg


In [11]:
#Create Mars dictionary
mars_dict=dict()

In [12]:
#Add Mars news title, paragraph, and image URL to dictionary
from pprint import pprint
mars_dict['Mars News Title']=news_title
mars_dict['Mars News Paragraph']=news_p
mars_dict['Mars Featured Image (URL)']=featured_image_url

## Mars Weather

In [13]:
#Visit Twitter; scrape latest Mars weather tweet
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

weather_url='https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
html=browser.html
soup_weather=BeautifulSoup(html, 'html.parser')

In [14]:
#Save tweet to variable, print
mars_weather=soup_weather.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
mars_weather

'Curiosity is again operating normally following a boot problem first experienced last Friday. Look for more Gale Crater weather conditions soon.\nhttps://www.jpl.nasa.gov/news/news.php?feature=7339\xa0…pic.twitter.com/gFMfXyeWDa'

In [15]:
#Add to Mars dictionary
mars_dict['Mars Weather']=mars_weather

## Mars Facts

In [16]:
#Visit Mars Facts site; scrape the table containing facts
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

facts_url='http://space-facts.com/mars/'
marsFacts_df=pd.read_html(facts_url)
marsFacts_df=marsFacts_df[0]
marsFacts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
#Rename columns 
marsFacts_df.columns = ['Mars Facts', 'Values']

#Use Pandas to convert data to HTML table string
marsFacts_df.to_html('mars_facts.html', index=False)

#Reset index
marsFacts_df.set_index('Mars Facts')

#New HTML version of Mars facts table
marsFacts_html = marsFacts_df.to_html(classes='mars_facts table table-striped')

In [18]:
#Add to Mars dictionary
mars_dict['Mars Facts Table']=marsFacts_html

## Mars Hemispheres

In [19]:
#Obtain hi-res images for each of Mars' hemispheres
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

hems_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hems_url)
time.sleep(5)
html=browser.html
soup_hems=BeautifulSoup(html,'html.parser')

In [20]:
#Parse soup object to get images of hemispheres 
class_collapse=soup_hems.find('div', class_='collapsible results')
images=class_collapse.find_all('div', class_='item')

In [21]:
#Loop to find title & image URLs
hem_urls=list()
img_urls=list()
titles=list()

for image in images:
    hem_title=image.h3.text
    titles.append(hem_title)
    
    image_href='https://astrogeology.usgs.gov'+image.find('a', class_='itemLink product-item')['href']
    
    browser.visit(image_href)
    time.sleep(5)
    html=browser.html
    soup_hems_img=BeautifulSoup(html, 'html.parser')
    hem_img_url=soup_hems_img.find('div', class_='downloads').find('li').a['href']
    print('Hem Image URL: '+ hem_img_url)
    img_urls.append(hem_img_url)
    
    #Create dictionary
    hems_dict=dict()
    hems_dict['title']=hem_title
    hems_dict['img_url']=hem_img_url
    
    hem_urls.append(hems_dict)

Hem Image URL: http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Hem Image URL: http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Hem Image URL: http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Hem Image URL: http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [22]:
#Print lists
print(hem_urls)
print(img_urls)
print(titles)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineri

In [23]:
#Add to Mars dictionary
mars_dict['Mars Hemisphere Image URLs']=hem_urls
#pprint(mars_dict)

In [24]:
#Print complete Mars dictionary
pprint(mars_dict)

{'Mars Facts Table': '<table border="1" class="dataframe mars_facts table '
                     'table-striped">\n'
                     '  <thead>\n'
                     '    <tr style="text-align: right;">\n'
                     '      <th></th>\n'
                     '      <th>Mars Facts</th>\n'
                     '      <th>Values</th>\n'
                     '    </tr>\n'
                     '  </thead>\n'
                     '  <tbody>\n'
                     '    <tr>\n'
                     '      <th>0</th>\n'
                     '      <td>Equatorial Diameter:</td>\n'
                     '      <td>6,792 km</td>\n'
                     '    </tr>\n'
                     '    <tr>\n'
                     '      <th>1</th>\n'
                     '      <td>Polar Diameter:</td>\n'
                     '      <td>6,752 km</td>\n'
                     '    </tr>\n'
                     '    <tr>\n'
                     '      <th>2</th>\n'
                     '      <